<a href="https://colab.research.google.com/github/tpgjs97/company-project/blob/master/%EC%A0%84%EC%B2%98%EB%A6%AC/smartnurse_%EC%A0%84%EC%B2%98%EB%A6%AC%EC%BD%94%EB%93%9C_%EC%84%B8%ED%97%8C_(%EB%91%90_df_%ED%95%A9%EC%B9%9C_%ED%9B%84_%EC%A4%91%EB%B3%B5%EC%A0%9C%EA%B1%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install nltk pandas konlpy

In [2]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (24.2 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/s

# 라이브러리 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import json
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# NLTK 리소스 다운로드
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 데이터 불러오기

In [358]:
import json
# ncp JSON 파일 열기
with open('/content/drive/MyDrive/ColabNotebooks/YearDreamSchool/FinalProject/Data/nursingrecord_ncp.json', 'r') as file:
    ncp_data = json.load(file)

# aws JSON 파일 열기
with open('/content/drive/MyDrive/ColabNotebooks/YearDreamSchool/FinalProject/Data/nursingrecord_aws.json', 'r') as file:
    aws_data = json.load(file)

In [359]:
# JSON 문자열을 파이썬 딕셔너리로 변환하는 함수
def parse_json(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return {}  # JSON 파싱 에러 시 빈 딕셔너리 반환

# 각 레코드에 대해 'content' 필드를 파싱
for item in ncp_data:
    item['content'] = parse_json(item['content'])

for item in aws_data:
    item['content'] = parse_json(item['content'])


# 데이터프레임 생성
ncp_df = pd.DataFrame(ncp_data)
aws_df = pd.DataFrame(aws_data)


# 전처리

In [360]:
import pandas as pd

# 'record_type'이 0인 행들만 필터링
ncp_record_type_0 = ncp_df[ncp_df['record_type'] == 0]
aws_record_type_0 = aws_df[aws_df['record_type'] == 0]

# 'content' 열을 확장하여 새로운 데이터 프레임 생성
ncp_content_df_0 = pd.json_normalize(ncp_record_type_0['content'])
aws_content_df_0 = pd.json_normalize(aws_record_type_0['content'])

# 필요한 열만 가져오기
ncp_content_df_0 = ncp_content_df_0[['domain', 'class', 'diagnosis', 'collectingData', 'goal', 'plan', 'interventions', 'evaluation']]
aws_content_df_0 = aws_content_df_0[['domain', 'class', 'diagnosis', 'collectingData', 'goal', 'plan', 'interventions', 'evaluation']]

In [361]:
# 두 데이터프레임 합치기
nanda = pd.concat([ncp_content_df_0, aws_content_df_0])

# 중복된 행 제거
nanda = nanda.drop_duplicates()
print(f'중복제거 전 column 개수 : {len(ncp_content_df_0) + len(aws_content_df_0)} / 중복제거 후 column 개수 : {len(nanda)}')

중복제거 전 column 개수 : 2552 / 중복제거 후 column 개수 : 1917


In [354]:
import pandas as pd
import re

# 필터링 조건에 맞지 않는 행을 확인하는 함수
def is_valid_row(text):
    # 숫자만 있는 경우
    if re.fullmatch(r'\d+', text):
        return False
    # 영어 알파벳만 있는 경우
    if re.fullmatch(r'[a-zA-Z]+', text):
        return False
    # 한글 자음 또는 모음만 있는 경우
    if re.fullmatch(r'[ㄱ-ㅎㅏ-ㅣ]+', text):
        return False
    # 특수 문자만 있는 경우
    if re.fullmatch(r'[^\w\s]+', text):
        return False
    return True

# 필터링 조건에 맞지 않는 단어를 제거하는 함수
def remove_invalid_words(text):
    if is_valid_row(str(text)) :
        valid_words = text
    else :
        valid_words = pd.NA # 빈 문자열 대신 pd.NA 반환
    return valid_words

# 조건에 맞는 단어로 대체하는 함수
def text_preprocessing(text):
    if text is not pd.NA :
        text = text.replace('\\n', ' ')
        text = text.replace('\n', ' ')
        txt = re.sub('[^가-힣a-zA-Z0-9./%]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
    else :
        txt = pd.NA
    return txt

# 조건을 만족하는 단어만 남깁니다.
for column in nanda.columns:
    nanda[column] = nanda[column].apply(remove_invalid_words)
    nanda[column] = nanda[column].apply(text_preprocessing)

# 모든 열이 비어 있는 행을 제거합니다.
nadna = nanda[~nanda.apply(lambda x: x.astype(str).str.contains('<NA>')).any(axis=1)]

# 결과 출력
nanda

,domain,class,diagnosis,collectingData,goal,plan,interventions,evaluation
0,안정/보호,체온조절,고체온,37.8도,체온이 37도 이하로 적용한,Ice Bag 적용,Ice Bag 적용,체온 36.8도로 떨어짐
1,안위,신체적 안위,만성 통증,아파요 NRS 10점 아파요 NRS 10점 ...,1. 대상자는,<NA>,<NA>,<NA>
2,안위,신체적 안위,만성 통증,숨쉴때 통증 호소/NRS 6점,통증 경감/호흡을 정상적으로 할 수 있다.,통증 경감,호흡시 통증 정도가 적은 자세를 취하도록함 통증이 심할시 담당의사에게 알려 필요한 ...,호전됨
3,건강증진,건강관리,비효율적 건강관리,숨쉴때 통증 호소/NRS 6점,통증 경감/호흡을 정상적으로 할 수 있다.,통증 감소를 위해 담당의 노티 파이,통증 감소를 위해 담당의 노티 파이,통증 감소함
4,자아지각,신체상,신체상 혼란,르 알 / 환청 호소함,환청을 감소시키기 위해 활동 격려,휴,<NA>,<NA>
...,...,...,...,...,...,...,...,...
1561,배설,비뇨기계 기능,긴장성 요실금,<NA>,74789,<NA>,<NA>,<NA>
1565,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1566,안위,신체적 안위,급성 통증,1. 주관적 자료 통증 호소 주호소 왼쪽 발뒤꿈치에 생긴 상처가 3주째 잘 낫...,간호목표 단기/장기 Goal 1. 목표 박씨는 퇴원 전 수용할 수 있는 통증 정도...,중재 계획 1. 대상자의 V/S를 1시간마다 사정한다. 진단적 중재 2. 2시간 ...,1. 활력징후 측정을 통해 통증으로 인한 호흡양상의 변화 등을 확인할 수 있다. ...,박씨에게 통증의 빈도와 지속시간에서 통증 조절의 목표가 달성되었는지 질문한다. 박씨...
1568,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [355]:
def filter_rows_by_length(df, max_length=4):
    # 각 열에 대해 글자 수를 계산하여 모든 열에 대한 조건을 만족하는 행 선택
    condition = df.apply(lambda col: col.apply(lambda x: len(str(x))) > max_length).all(axis=1)
    df_filtered = df[condition]

    return df_filtered

nanda_filtered = filter_rows_by_length(nanda)

# 결과 출력
nanda_filtered

,domain,class,diagnosis,collectingData,goal,plan,interventions,evaluation
38,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,S 기침이랑 가래가 계속 나와요 가끔씩 숨 쉬는게 힘들어요 O bron...,8시간내에 가래를 배출할 수 있다. 대상자는 7일 내에 가래 증상이 완화될 ...,Mucopect 30mg/tab tid Cetilan cap 200mg/tab ti...,처방된 약을 하루에 3번 투약한다. 부작용이 나타나는지 관찰한다. 산소가 잘 들어가...,라운딩 시 그렁거림이 줄어들었다.
41,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,주관적 숨이 가쁘고 가슴이 답답해요. 객관적 SPO2 94%,장기 목표 대상자는 호흡을 편안하게 할 수 있다. 단기 목표 일주일 이내에 대상...,1. 8시간 마다 대상자의 호흡음을 사정한다. 2. 8시간 마다 대상자의 spo2 ...,1. 그르렁거리는 소리 Coarse 와 wheezing sound는 폐렴과 좁아진...,1. 호흡음에서 점차 그르렁거리는 소리가 줄어들었다. 2. 산소포화도가 점차 정상 ...
53,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,S 숨 차요 O Sp02 90% RR 26 ABGA 추가 ...,1시간 이내 대상자의 Spo2가 target 범위를 유지한다. 대상자가 1일 이내...,1. V/S 특히 호흡수와 산소포화도를 시간마다 관찰한다. 오더에 따라서 ...,1. 호흡과 산소화 정도를 관찰하는 것은 대상자의 상태를 파악하는데 중요한 지표이다...,Spo2 결과
59,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,주관적 자료 숨을 쉬기가 너무 힘들어요. 기침이 계속해서 나와요....,단기목표 대상자는 2시간 이내 숨찬 느낌이 덜하다고 표현한다. 장기목표 ...,1. 1시간마다 V/S SpO2를 사정한다. 2. 대상자의 호흡 수 호흡양상 ...,1. 대상자의 전반적 상태를 알 수 있는 지표이다. 빈호흡 빈맥 낮은 SpO2수...,단기목표 대상자는 2시간 이내 숨찬 느낌이 덜하다고 표현하였다. 달성 ...
69,활동/휴식,심혈관/호흡기계 반응,비효과적 뇌조직 관류의 위험,주관적 자료 머리가 아프고 너무 어지러워요. 잠시동안 기억...,단기목표 대상자는 5시간 이내로 맥박과 의식이 정상적으로 유지된다. 장기목...,1. 1시간마다 대상자의 V/S를 사정한다. 2. 기도유지를 위해 침상 옆에 드레싱...,1. 두개 내압 상승으로 인한 cushing triad 수축기 혈압 상승 서맥 ...,단기목표 대상자는 5시간 이내로 맥박과 의식이 정상적으로 유지된다. 달성 ...
...,...,...,...,...,...,...,...,...
1519,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,주관적 보호자 아기가 숨쉬기 힘들어 보여요 객관적 chest retracti...,단기 24시간 동안 환아의 호흡수가 정상범위 내로 유지된다. 24시간 동안 환아의 ...,적용중인 O2를 처방에 따라 3L로 증량한다. 보호자에게 산소 주입 시 주의사항을 ...,처방에 따라 비강캐뉼라를 이용하여 O2 3L/min으로 공급함. 산소공급 시 유의점...,24시간 동안 환아의 호흡수가 정상범위 내로 유지된다. 달성 호흡수 50회/...
1520,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,S 보호자 아기가 숨쉬기 힘들어 보여요. O Sp02 88% 호흡수 66회/분 ...,단기목표 2시간 이내로 환아의 산소포화도 호흡수가 정상으로 돌아오며 호흡곤란 양...,처방대로 산소요법 비강캐뉼라 2L에서 3L로 증량,처방대로 산소요법 비강캐뉼라 2L에서 3L로 증량함,2시간 후 환아의 산소포화도 95% 호흡수55회/분으로 정상으로 돌아왔고 호흡곤란...
1521,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,S 보호자 아기가 숨 쉬기 힘들어해요. O chest retractio...,단기목표 환아의 spO2가 95이상으로 유지한다. 환아의 호흡수가 정상범위를...,산소포화도 1시간 마다 확인한다. nasal cannula로 산소를 3L 공...,산소포화도 1시간 마다 확인 시 95% 이상 달성한 것을 확인했다. nasa...,단기목표 환아의 spO2가 95이상으로 유지한다. 30분 뒤 spO2가 9...
1522,활동/휴식,심혈관/호흡기계 반응,비효과적 호흡 양상,S 보호자 아기가 숨 쉬기 힘들어해요. O chest retraction 비...,단기목표 환아의 spO2가 95이상으로 유지한다. 환아의 호흡수가 정상범위를 유지한...,산소포화도 1시간 마다 확인한다. nasal cannula로 산소를 3L 공급한다....,산소포화도 1시간 마다 확인 시 95% 이상 달성한 것을 확인했다. nasal ca...,단기목표 환아의 spO2가 95이상으로 유지한다. 30분 뒤 spO2가 96%...
